模型可以请求调用工具来执行任务，例如从数据库中获取数据、搜索网络或运行代码。工具是以下两项的配对：
- 一个模式（Schema），包含工具的名称、描述和/或参数定义（通常是 JSON 模式）。
- 一个用于执行的函数或协程。

为了让模型能够使用您定义的工具，必须使用 bind_tools 将其绑定。在后续的调用中，模型可以根据需要选择调用任何一个已绑定的工具。

后面的课程，我们会单独介绍工具调用的相关内容，这里只是简单介绍一下工具调用的基本流程，**需要明确的是。当你单独使用模型（而非通过框架）时，你需要自行执行该工具，并将结果返回给模型，供其在后续推理中使用。而当你使用封装过的框架时，框架会自动处理整个工具调用循环。**

In [21]:
from dotenv import load_dotenv
import os

load_dotenv()  # 自动查找当前目录的 .env 文件

dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")
dashscope_base_url = os.getenv("DASHSCOPE_BASE_URL")

In [22]:
from langchain_qwq import ChatQwen

model = ChatQwen(
    model="qwen-plus",
    api_key=dashscope_api_key,
    base_url=dashscope_base_url,
    temperature=0.7
)

# 本地工具调用

## 创建工具

创建工具最简单的方法是使用LangChain提供的@tool装饰器。默认情况下，函数的文档字符串会成为该工具的描述，帮助模型理解何时应使用该工具。

类型提示定义了工具的输入结构。文档字符串应信息明确且简洁，以帮助模型理解工具的用途。

In [23]:
def get_weather(location: str) -> str:
    """获取指定地点的天气。"""
    return f"{location} 的天气是 20 摄氏度，晴朗。"

## 绑定工具到模型

In [24]:
model_with_tools = model.bind_tools([get_weather])

## 调用模型

In [ ]:
response = model_with_tools.invoke("宁波的天气怎么样？")
print(response)

In [26]:
# 遍历模型返回的工具调用信息
for tool_call in response.tool_calls:
    print(f"工具名称: {tool_call['name']}")
    print(f"参数: {tool_call['args']}")
    print(f"调用ID: {tool_call['id']}")

工具名称: get_weather
参数: {'location': '宁波'}
调用ID: call_b78416c60c9344448784fe


如果是流式响应，需要注意的是工具调用的JSON参数是逐字生成的（如 {"location": "宁波"}），LangChain会逐步拼接这些片段，在完整JSON构造完成前，LangChain 会将中间片段标记为invalid_tool_calls，因为此时参数还不完整、无法解析。

# 服务端工具的调用

通常指模型提供商（如 OpenAI, Anthropic, Google 等）在他们自己的服务器端内置并提供的工具或函数调用能力。

In [10]:
model = ChatQwen(
    model="qwen-plus",
    api_key=dashscope_api_key,
    base_url=dashscope_base_url,
    temperature=0.7,
    # 开启工具调用，每个模型供应商都有自己定义的参数，请自行查看文档
    extra_body={"enable_search": True})

response = model.invoke("宁波明天天气如何")

In [ ]:
response.content_blocks